In [1]:
stock = 'CSCO'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-23    41.777546
2018-07-24    42.105328
2018-07-25    42.870159
2018-07-26    43.237671
2018-07-27    42.284119
2018-07-30    41.886803
2018-07-31    42.006001
2018-08-01    41.578888
2018-08-02    42.244389
2018-08-03    42.542374
2018-08-06    43.009216
2018-08-07    43.287338
2018-08-08    43.545589
2018-08-09    43.485992
2018-08-10    43.485992
2018-08-13    43.456196
2018-08-14    43.704517
2018-08-15    43.565456
2018-08-16    44.856728
2018-08-17    45.561958
2018-08-20    45.909611
2018-08-21    45.472561
2018-08-22    45.681152
2018-08-23    45.710953
2018-08-24    46.008938
2018-08-27    46.277122
2018-08-28    46.634705
2018-08-29    47.161144
2018-08-30    46.833363
2018-08-31    47.449200
                ...    
2018-09-18    47.141281
2018-09-19    46.962490
2018-09-20    47.409466
2018-09-21    48.233894
2018-09-24    48.114697
2018-09-25    48.144501
2018-09-26    48.084900
2018-09-27    48.005440
2018-09-28    48.323292
2018-10-01    48.541809
2018-10-02 

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


47.62723310115166
[49.49172378 48.34633195 46.41930714 47.93135982 45.94744281]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")